In [1]:
import os 
import json
import pandas as pd
import seaborn as sns
import numpy as np
import math

from utils.analysys_utils import list_experiments_in_dir

In [2]:
experiments_path = os.path.join(os.getcwd(), "derivative_reg", "sidarthe_extended", "Italy")
experiments = list_experiments_in_dir(experiments_path)

# sort experiments by derivative regularization term
experiments.sort(key=lambda exp: exp['settings']['der_1st_reg'])

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'd:\\GitHub\\covid-tools\\derivative_reg\\sidarthe_extended\\Italy'

In [3]:
# generate data for graphs

# take as baseline the exp with 0. regularization
base_val_loss = experiments[0]['final']['best_epoch']['val_loss']
base_der_loss = 0.

# use value of derivative regularization as x axis
x_axis = []
val_losses = []
der_losses = []
indices = []
for exp in experiments:
    der_1st_reg = exp['settings']['der_1st_reg']
    if der_1st_reg == 0.:
        continue

    x_axis.append(der_1st_reg)

    val_loss = exp['final']['best_epoch']['val_loss']
    val_losses.append(val_loss)

    # normalize w.r.t. regularization weight
    der_1st_loss = exp['final']['best_epoch']['losses']['der_1st'] / der_1st_reg
    der_losses.append(der_1st_loss)

    val_gain = base_val_loss - val_loss
    indices.append(val_gain / -math.log10(der_1st_loss))


data = {"der_reg": x_axis, "validation": val_losses, "derivative": der_losses, "index": indices}

data["index"]

NameError: name 'experiments' is not defined

In [4]:
# plot regularization weight vs validation loss
plot = sns.lineplot(data=data, x="der_reg", y="validation")
plot.set(xscale="log", yscale="log")
plot.set_title("Regularization Weight vs Validation Loss")

NameError: name 'data' is not defined

In [5]:
# plot regularization weight vs derivative loss
plot = sns.lineplot(data=data, x="der_reg", y="derivative")
plot.set_title("Regularization Weight vs Derivative Loss")
plot.set(xscale="log", yscale="log")

NameError: name 'data' is not defined

In [6]:
# draw scatter plot to see pareto front

